In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import distance as dist

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)


mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)


LEFT_EYE_IDX = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_IDX = [362, 385, 387, 263, 373, 380]

EAR_THRESHOLD = 0.22
EAR_FRAMES_REQUIRED = 2
blink_counter = 0
tears = []  

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb)

    if result.multi_face_landmarks:
        face = result.multi_face_landmarks[0]

        left_eye = np.array([(int(face.landmark[i].x * w), int(face.landmark[i].y * h)) 
                             for i in LEFT_EYE_IDX])
        right_eye = np.array([(int(face.landmark[i].x * w), int(face.landmark[i].y * h)) 
                              for i in RIGHT_EYE_IDX])

        
        left_EAR = eye_aspect_ratio(left_eye)
        right_EAR = eye_aspect_ratio(right_eye)
        EAR = (left_EAR + right_EAR) / 2

       
        if EAR < EAR_THRESHOLD:
            blink_counter += 1
        else:
            if blink_counter >= EAR_FRAMES_REQUIRED:
                lx, ly = int(np.mean(left_eye[:,0])), int(np.mean(left_eye[:,1])) + 15
                rx, ry = int(np.mean(right_eye[:,0])), int(np.mean(right_eye[:,1])) + 15
                tears.append([lx, ly])
                tears.append([rx, ry])
            blink_counter = 0


    new_tears = []
    for (x, y) in tears:
        cv2.circle(frame, (x, y), 10, (255, 200, 100), -1)  # gota color azul pastel
        y += 6  # velocidad de caída
        if y < h - 10:
            new_tears.append([x, y])
    tears = new_tears

    cv2.imshow("Filtro Parpadeo con Lágrimas (MediaPipe)", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


NameError: name 'audio_classifier' is not defined